<a href="https://colab.research.google.com/github/abishekabi/Volcanic-Eruption-Prediction/blob/main/predict_volcanic_eruptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks/predict-volcanic-eruptions'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [72]:
%cd 'gdrive/My Drive/Colab Notebooks/predict-volcanic-eruptions'

[Errno 2] No such file or directory: 'gdrive/My Drive/Colab Notebooks/predict-volcanic-eruptions'
/content/gdrive/My Drive/Colab Notebooks/predict-volcanic-eruptions


In [73]:
# !wget "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/19059/1581061/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1603216878&Signature=Xcz3Ni8%2F7bb7rIyQlyeyfbeASoi78143MpaVMe3kyZXi3WKfxcW7kp%2F44CQYzgoE2FfwzGR343oh5QVkHTt9xTGjHufJZRxbiJ0allxJiMFJrdZaZLZP8UwcQGl4k7ZlyF%2Bc7lAcoYNI14LjW4GEUSTOYvWyIrJqWGsBPwYCcLaGLaHP4lKxF0W7M%2FIOlyIMASugpiRIVsuCkR8fNWT3k4vyRgSldLTlfI46fkNvOFIsOl8kU%2BeRVd8yuhBrA8EmY8QRR0%2FGl9wSCfyjQe8Or4nSWs5V5svpRrzbtl5gTeMcIi8emEAktsbg8lK3%2F%2BzpIMu4map2I1aXj1wAqMPnYA%3D%3D&response-content-disposition=attachment%3B+filename%3Dpredict-volcanic-eruptions-ingv-oe.zip"
# !unzip data.zip
# from google.colab import files
# files.upload() 
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !ls ~/.kaggle
# !chmod 600 /root/.kaggle/kaggle.json  # set permission
#!kaggle competitions download -c predict-volcanic-eruptions-ingv-oe 

In [74]:
!ls -lh 

total 9.5G
-rw------- 1 root root 9.5G Oct 17 18:06 data.zip
-rw------- 1 root root  59K Oct 27 23:50 predict-volcanic-eruptions.ipynb
-rw------- 1 root root  56K Oct 17 18:08 sample_submission.csv
drwx------ 2 root root 4.0K Oct 17 18:06 test
drwx------ 2 root root 4.0K Oct 19 15:53 train
-rw------- 1 root root  84K Oct 19 16:30 train.csv


In [75]:
import gc
import glob
from itertools import chain

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [76]:
 df=pd.read_csv('train.csv')
df.head()

,segment_id,time_to_eruption
0,1136037770,12262005
1,1969647810,32739612
2,1895879680,14965999
3,2068207140,26469720
4,192955606,31072429


In [77]:
df=pd.read_csv('train/1136037770.csv')
df.head()

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10
0,220.0,339.0,-336.0,364.0,NaN,492.0,-796.0,-423.0,-582.0,-40.0
1,178.0,221.0,-317.0,366.0,NaN,525.0,-754.0,-415.0,-932.0,-305.0
2,151.0,42.0,-280.0,250.0,NaN,463.0,-772.0,-229.0,-257.0,-59.0
3,162.0,-123.0,-243.0,288.0,NaN,303.0,-899.0,212.0,-295.0,38.0
4,158.0,-287.0,-300.0,372.0,NaN,169.0,-769.0,755.0,169.0,-239.0


In [78]:
df=pd.read_csv('sample_submission.csv')
df.head()

,segment_id,time_to_eruption
0,1000213997,0
1,100023368,0
2,1000488999,0
3,1001028887,0
4,1001857862,0


In [79]:
df = pd.read_csv('test/1000213997.csv')
df.head()

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10
0,180.0,NaN,295.0,-221.0,-81.0,379.0,240.0,-288.0,152.0,0.0
1,200.0,NaN,199.0,-99.0,32.0,-226.0,241.0,-114.0,36.0,0.0
2,193.0,NaN,113.0,17.0,164.0,52.0,294.0,27.0,-21.0,0.0
3,188.0,NaN,26.0,125.0,274.0,-278.0,324.0,338.0,-22.0,0.0
4,177.0,NaN,-48.0,183.0,355.0,-363.0,356.0,339.0,-32.0,0.0


In [80]:
!ls

data.zip			  sample_submission.csv  train
predict-volcanic-eruptions.ipynb  test			 train.csv


# Data

In [81]:
train = pd.read_csv("train.csv")
sample_submission = pd.read_csv("sample_submission.csv")


In [82]:
train_segment_csvs = glob.glob("train/*")
test_segment_csvs = glob.glob("test/*")

In [83]:
train_segment_csvs[0:5]

['train/579307667.csv',
 'train/579908624.csv',
 'train/579683001.csv',
 'train/580262567.csv',
 'train/581069262.csv']

In [84]:
test_segment_csvs[0:5]

['test/58542575.csv',
 'test/585632599.csv',
 'test/585794453.csv',
 'test/585631329.csv',
 'test/587942731.csv']

In [85]:
print("Number of files under train/ is:", len(train_segment_csvs))
print("Number of files under test/ is:", len(test_segment_csvs))

duplicated_segment_id = [segment_id for segment_id in [ test_segment_id.split("/")[-1] for test_segment_id in test_segment_csvs]
                         if (segment_id in [ train_segment_id.split("/")[-1] for train_segment_id in train_segment_csvs])]

print("Duplicates Segment ids both in train and test are:", len(duplicated_segment_id))

Number of files under train/ is: 4431
Number of files under test/ is: 4520
Duplicates Segment ids both in train and test are: 0


In [86]:
train['time_to_eruption'].describe()

count    4.431000e+03
mean     2.284891e+07
std      1.348439e+07
min      6.250000e+03
25%      1.127016e+07
50%      2.246559e+07
75%      3.434356e+07
max      4.904609e+07
Name: time_to_eruption, dtype: float64

## Load Data

In [87]:
column_names = ["sensor_1", 	"sensor_2", 	"sensor_3", 	"sensor_4", 	"sensor_5", 	"sensor_6", 	"sensor_7", 	"sensor_8", 	"sensor_9", 	"sensor_10"]
column_stats = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
columns = [ i+"_"+j for i in column_names for j in column_stats]
X = pd.DataFrame(columns=columns)
y = pd.DataFrame(columns=['time_to_eruption'])
# display(X.columns)
# train_csv = pd.read_csv(train_segment_csvs[0])
for i, train_segment_csv in train[:10].iterrows():
#for train_segment_csv in train_segment_csvs[:10]:
  # train_csv = pd.read_csv(train_segment_csv)
  train_csv = pd.read_csv("train/"+ str(train_segment_csv.segment_id) + ".csv")
  y.loc[len(X), :] = train_segment_csv.time_to_eruption
  row = []
  for feature in train_csv.columns:
    row.extend(train_csv[feature].describe().values)
  X.loc[len(X), :] = row


In [88]:
display(X.head())

,sensor_1_count,sensor_1_mean,sensor_1_std,sensor_1_min,sensor_1_25%,sensor_1_50%,sensor_1_75%,sensor_1_max,sensor_2_count,sensor_2_mean,sensor_2_std,sensor_2_min,sensor_2_25%,sensor_2_50%,sensor_2_75%,sensor_2_max,sensor_3_count,sensor_3_mean,sensor_3_std,sensor_3_min,sensor_3_25%,sensor_3_50%,sensor_3_75%,sensor_3_max,sensor_4_count,sensor_4_mean,sensor_4_std,sensor_4_min,sensor_4_25%,sensor_4_50%,sensor_4_75%,sensor_4_max,sensor_5_count,sensor_5_mean,sensor_5_std,sensor_5_min,sensor_5_25%,sensor_5_50%,sensor_5_75%,sensor_5_max,sensor_6_count,sensor_6_mean,sensor_6_std,sensor_6_min,sensor_6_25%,sensor_6_50%,sensor_6_75%,sensor_6_max,sensor_7_count,sensor_7_mean,sensor_7_std,sensor_7_min,sensor_7_25%,sensor_7_50%,sensor_7_75%,sensor_7_max,sensor_8_count,sensor_8_mean,sensor_8_std,sensor_8_min,sensor_8_25%,sensor_8_50%,sensor_8_75%,sensor_8_max,sensor_9_count,sensor_9_mean,sensor_9_std,sensor_9_min,sensor_9_25%,sensor_9_50%,sensor_9_75%,sensor_9_max,sensor_10_count,sensor_10_mean,sensor_10_std,sensor_10_min,sensor_10_25%,sensor_10_50%,sensor_10_75%,sensor_10_max
0,60001,-1.61032,303.096,-1421,-199,0,199,1353,60001,4.61382,678.331,-5199,-391,0,403,5572,60001,3.55972,290.873,-1453,-184,0,187,1413,60001,2.01998,384.141,-1960,-254,0,258,1699,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60001,-9.71885,696.936,-2776,-481,0,475,2432,60001,7.06003,521.11,-2338,-344,0,352,2969,60001,1.26836,620.95,-2878,-423,0,426,2544,60001,1.46504,372.387,-1747,-242,0,245,2021,60001,2.59944,596.866,-3527,-406,0,410,2187
1,60001,1.42613,438.361,-2005,-265,0,270,2197,60001,2.48444,661.163,-4267,-381,0,381,5111,60001,-2.79427,362.2,-2094,-224,0,221,1723,60001,-1.70057,369.483,-2166,-229,0,229,1923,60001,0.732104,274.668,-2090,-153,0,154,1764,60001,-8.97507,360.132,-1684,-227,0,223,1888,60001,5.87022,475.776,-3549,-275,0,292,2442,60001,-1.08683,364.99,-1716,-236,0,242,1476,59263,3.17923,407.418,-2615,-246,0,255,2189,60001,19.8955,932.7,-4591,-547,0,565,5319
2,60001,2.50459,241.575,-1115,-157,0,157,1148,60001,5.44971,831.469,-6147,-455,0,467,7587,60001,-1.58854,217.987,-1207,-136,0,137,1159,60001,-1.15083,293.571,-1816,-188,0,185,1583,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60001,-13.3284,468.816,-1812,-329,0,308,1564,60001,-1.95763,356.338,-1481,-232,0,229,1601,60001,-4.11161,350.322,-1293,-238,0,238,1202,60001,-1.83334,261.36,-1542,-161,0,159,1778,60001,0.968101,500.545,-2798,-323,0,321,2522
3,60001,2.1658,221.968,-1025,-137,0,140,1254,60001,-0.373377,425.19,-2828,-253,0,252,2453,60001,-2.69899,194.087,-847,-127,0,123,1345,60001,-1.31686,239.593,-1102,-157,0,152,1055,60001,0.308795,167.917,-1693,-98,0,98,1166,60001,-0.836886,311.503,-1840,-204,0,202,2025,60001,-0.843136,292.563,-1533,-185,0,185,1576,60001,-0.175314,321.893,-1457,-214,0,214,1542,57477,-3.51092,237.985,-1574,-154,0,150,1340,60001,8.38144,523.45,-2911,-323,0,326,3614
4,60001,0.0738154,261.696,-1190,-171,0,170,1114,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60001,1.99888,227.924,-1184,-149,0,152,1176,60001,0.245079,262.59,-1308,-174,0,171,1290,60001,-0.441376,190.745,-1052,-118,0,117,1060,60001,2.20565,445.919,-1715,-302,0,312,1700,60001,-4.78435,312.576,-1103,-224,0,213,1036,60001,-1.73032,412.606,-1618,-284,0,282,1439,60001,1.62291,325.089,-7252,-166,0,168,7725,60001,-3.74047,562.748,-2611,-362,0,361,3274


In [89]:
y

,time_to_eruption
0,12262005
1,32739612
2,14965999
3,26469720
4,31072429
5,22264866
6,34952168
7,9504818
8,11719563
9,11629084


### Cleanup NaN

In [90]:
# Try Different Methods like kNN
X = X.fillna(0)

## Split Data to train & test

In [91]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

## Train 

In [92]:
reg = LinearRegression().fit(X_train, y_train)
#reg.score(X, y)

#Test

In [93]:
from sklearn.metrics import mean_squared_error, r2_score

In [94]:
y_pred = reg.predict(X_test)

In [95]:
# The coefficients
print('Coefficients: \n', reg.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))

Coefficients: 
 [[ 2.24531505e-12  2.30706842e+00 -6.07105740e+01 -2.35173029e+01
   7.64452774e+01  0.00000000e+00 -7.37874817e+01  2.35043008e+02
   3.29498848e+01  2.37461378e+00 -1.64527242e+02  2.65703706e+03
   1.34629759e+02  0.00000000e+00 -1.36267912e+02 -1.57228551e+02
   0.00000000e+00 -3.83576331e+00 -2.01795017e+02 -1.05512225e+01
   1.53617516e+02  0.00000000e+00 -1.68364289e+02 -1.23093582e+02
   0.00000000e+00  3.91245080e+00 -1.63762273e+02  2.01725362e+02
   1.29657714e+02  0.00000000e+00 -1.23106357e+02 -4.04718969e+02
  -5.21334185e+01 -3.08059482e-01  8.87241668e+01 -6.15138330e+02
  -3.20490528e+01  0.00000000e+00  2.90344619e+01  5.46278569e+02
   0.00000000e+00 -1.31645720e+01 -1.95461223e+02  2.11260726e+03
   1.15638497e+02  0.00000000e+00 -1.26697120e+02 -1.91572533e+03
   0.00000000e+00  7.01227000e+00  1.87328162e+02 -1.11212411e+03
  -9.41275299e+01  0.00000000e+00  1.16752488e+02 -1.29509831e+03
   0.00000000e+00 -3.39206787e+00 -6.91420945e+01  1.0358226

In [96]:
print(y_pred, y_test)

[[15012650.1187871 ]
 [10566931.80950937]
 [20379540.1135169 ]
 [ 9147234.74663575]]   time_to_eruption
9         11629084
0         12262005
5         22264866
6         34952168


In [97]:
print(X_train.shape, y_train.shape)

(6, 80) (6, 1)


In [98]:
# # Plot outputs
# plt.scatter(X_train, y_train,  color='black')
# plt.plot(X_test, y_pred, color='blue', linewidth=3)

# plt.xticks(())
# plt.yticks(())

# plt.show()

In [99]:
# train_csv = pd.read_csv(train_segment_csvs[0])
# display(train_csv.sensor_1.describe())
# for i, row in train[:2].iterrows():
#   print("Segment ID: %d \t Time to Eruption: %d" %(row.segment_id, row.time_to_eruption) )
#   sensor_data = pd.read_csv("train/"+ str(row.segment_id) + ".csv")
#   #sensor_data = sensor_data.fillna(0)
#   #sensor_data = sensor_data.dropna(axis=1)
#   print(sensor_data.isna().sum())
#   with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(sensor_data.describe())
#   print(sensor_data.info())
#   print("--"*30)
# sensor_data.describe().unstack()
# def calculate_stats(sensor_row):
#   X = pd.DataFrame()
#   for row in sensor_row:
#     out = sensor_row[row].describe().values
#     print(out) 
          
# # display(train_csv.sensor_1.describe().values)
# #   print(sensor_row.describe())
#   #X = pd.DataFrame()
#   # sensor_row.mean()
#   # input_data.sum()
#   # X.loc[ts, f'{sensor_id}_mean']      = signal.mean()
#   # X.loc[ts, f'{sensor_id}_std']       = signal.std()
#   # X.loc[ts, f'{sensor_id}_var']       = signal.var() 
#   # X.loc[ts, f'{sensor_id}_max']       = signal.max()
#   # X.loc[ts, f'{sensor_id}_min']       = signal.min()

# # def basic_statistics(, sensor_id, s):
# #     """Computes basic statistics for the training feature set.
    
# #     Args:
# #         t_X (pandas.DataFrame): The feature set being built.
# #         x (pandas.Series): The signal values.
# #         s (int): The integer number of the segment.
# #         postfix (str): The postfix string value.
# #     Return:
# #         t_X (pandas.DataFrame): The feature set being built.
# #     """
# #     X = pd.DataFrame()
# #     X.loc[sensor_num, f'{sensor_id}_mean']      = signal.mean()
# #     X.loc[sensor_num, f'{sensor_id}_std']       = signal.std()
# #     X.loc[sensor_num, f'{sensor_id}_var']       = signal.var() 
# #     X.loc[sensor_num, f'{sensor_id}_max']       = signal.max()
# #     X.loc[sensor_num, f'{sensor_id}_min']       = signal.min()

# #     X.loc[s, f'{sensor}_sum{postfix}']       = x.sum()
# #     t_X.loc[s, f'{sensor}_mean{postfix}']      = x.mean()
# #     t_X.loc[s, f'{sensor}_std{postfix}']       = x.std()
# #     t_X.loc[s, f'{sensor}_var{postfix}']       = x.var() 
# #     t_X.loc[s, f'{sensor}_max{postfix}']       = x.max()
# #     t_X.loc[s, f'{sensor}_min{postfix}']       = x.min()
# #     t_X.loc[s, f'{sensor}_median{postfix}']    = x.median()
# #     t_X.loc[s, f'{sensor}_skew{postfix}']      = x.skew()
# #     t_X.loc[s, f'{sensor}_mad{postfix}']       = x.mad()
# #     t_X.loc[s, f'{sensor}_kurtosis{postfix}']  = x.kurtosis()

# #     return t_X
# for i, row in train[:1].iterrows():
#   print("Segment ID: %d \t Time to Eruption: %d" %(row.segment_id, row.time_to_eruption) )
#   sensor_data = pd.read_csv("train/"+ str(row.segment_id) + ".csv")
#   print(calculate_stats(sensor_data))
#   #sensor_data = sensor_data.fillna(0)
#   #sensor_data = sensor_data.dropna(axis=1)
#   # print(sensor_data.isna().sum())
#   # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#   #   print(sensor_data.describe())
#   # print(sensor_data.info())
#   print("--"*30)
# train
# # train_379022420 = pd.read_csv("train/379022420.csv")
# # train_1002275321 = pd.read_csv("train/1002275321.csv")
# train.head()
# train.info()

# train_final = pd.DataFrame(columns={'segment_id', 'time_to_eruption', 'sensor_1', 'sensor_2', 'sensor_3',
#        'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9',
#        'sensor_10'})
# for i, row in train[0:20].iterrows():
#   sensor_data = pd.read_csv("train/"+ str(row.segment_id) + ".csv")
#   sensor_data['segment_id'] = row.segment_id
#   train1 = train.merge(sensor_data, on='segment_id')
#   # train_1_describe = pd.DataFrame(train1.describe()).reset_index()
#   # train_1_describe['segment_id'] = row.segment_id
#   train_final = pd.concat([train_final, train1], axis=0)
# train_final = train_final.reset_index(drop=True)
# train_final.groupby(['segment_id', 'time_to_eruption']).mean()
# train_final.isna().sum()

In [100]:
for train_segment_csv in train_segment_csvs[0:1]:
  train_csv = pd.read_csv(train_segment_csv)
  display(train_csv.sensor_1.describe().keys())

Index(['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'], dtype='object')